<h3><strong>Описание задачи <strong><h3>

<p>В этом проекте мы реализуем&nbsp;механизм подсказок (<em>suggest</em>) категории по заголовку в объявлениях Авито.</p>

<p>Представьте, что вы хотите разместить объявление о продаже шин. Чтобы положить своё объявление в нужную категорию, вам необходимо выбрать одну из 10 верхнеуровневых категорий, одну из 5 на следющем уровне, одну из 12 на следующем уровне, одну из 5 на последнем уровне.</p>

<p>Даже если варианты правильных&nbsp;категорий&nbsp;на каждом уровне очевидны, процесс подачи объявления лучше упростить.</p>

<p>&nbsp;Упрощённый процесс подачи объявлений может выглядеть следующим образом. Пользователь заполняет заголовок объявления, затем выбирает один из предложенных вариантов категорий, который больше всего подходит его объявлению.</p>

<p>Задача &mdash; определить категорию объявления, используя заголовок. Всего в выборке будет 54 категории.</p>


<h3><strong>Многоклассовая классификация</strong></h3>

<p>Для решения задачи будем использовать модель логистической регрессии – линейная модель для бинарной классификации. Как её использовать для решения задачи классификации на 54 класса? Оказывается, любой бинарный классификатор, который способен оценивать меру принадлежности объектов к одному из 2 классов, можно использовать для многоклассовой классификации. </p>

<p>Мнгоклассовая классификация на основе бинарного классификатора может быть реализована с помощью одной из двух стратегий:</p>

<ul>
	<li>Один против всех (<em>One-vs-Rest</em> или <em>One-vs-All</em>);</li>
	<li>One-vs-One.</li>
</ul>

<p> </p>

<p><strong>One-vs-All </strong></p>

<p>В стратегии <em>One-vs-All</em> строятся k</span> классификаторов, отделяющих каждый класс от остальных. Например, для 3 классов нужно обучить 3 классификатора:</p>

<div style="text-align: center;">
<figure class="image" style=""><img alt="" height="436" name="Снимок экрана 2021-08-03 в 19.26.56.png" src="https://ucarecdn.com/2abe5812-3f6e-42cf-97a4-a35a6de58c6d/" width="513">
<figcaption>Многоклассовая классфикация по стратегии OVR</figcaption>
</figure>
</div>

<p>Помимо эффективности построения (необходимы только k классификаторов), одним из преимуществ этого подхода является его интерпретируемость. Поскольку для каждого класса обучался свой классификатор, можно анализировать каждый классификатор в отдельности, чтобы понять, почему модель приняла такое решение.</p>

<p><em>Предсказания на тестовой выборке для OVR.</em></p>

<p>Для объектов из тестовой выборки предсказывается класс, получивший максимальную вероятность в результате предсказаний <span class="math-tex">\(k\)</span> классификаторов:</p>
$$
\hat{y} = \underset{k}{\arg\max} \ a_{k}(x),
$$
<p>где <span class="math-tex">\(a_k(x) -\)</span> предсказание модели бинарной классификации для класса <span class="math-tex">\(k\)</span>.  Выражение<em> <em><em><em><em><em><em><em><span class="math-tex">\(\argmax\)</span></em></em></em></em></em></em></em></em> означает выбор <span class="math-tex">\(k\)</span>, при котором значение <span class="math-tex">\(a_{k}(x)\)</span> максимально.</p>

<p><em>На практике обычно используют OVR стратегию или алгоритмы с нативной поддержкой многоклассовой классификации – нейронные сети или алгоритмы, которые мы будем изучать дальше в этом курсе (RandomForest, KNN).</em></p>

<p> </p>

<p><strong>One-vs-One (опционально)</strong></p>

<p>В стратегии <em>One-vs-One </em>обучается один классификатор для каждой пары классов. При этом каждый классификатор обучается только на объектах, относящихся к этим классам: </p>

<div style="text-align: center;">
<figure class="image" style=""><img alt="" height="551" name="Снимок экрана 2021-08-01 в 23.43.15.png" src="https://ucarecdn.com/b6a950d0-a787-49de-b6cd-9f2faadb91c3/" width="637">
<figcaption>Многклассовая классификация стратегией OVO</figcaption>
</figure>
</div>

<p>Таким образом, требуется построить квадратичное число классификаторов <span class="math-tex">\(\frac{k(k-1)}{2}\)</span>.</p>

<p>При больших <span class="math-tex">\(k\)</span> может потребоваться много времени на обучение. Однако этот метод может быть полезен для алгоритмов, которые плохо масштабируются с увеличением размера выборки, так как каждая отдельная задача обучения классификатора включает в себя только объекты, принадлежащие двум выбранным классам. </p>

<p><em>Предсказания на тестовой выборке для OVO.</em></p>

<p>Для объектов из тестовой выборки все <span class="math-tex">\(\frac{k(k-1)}{2}\)</span> классификаторов делают предсказания, а финальный ответ получается в результате «жёсткого» голосования:</p>

<p style="text-align: center;"><span class="math-tex">\(\hat{y} = \underset{k}{argmax} \sum_{m \neq k} [a_{km}(x) = 1],\)</span></p>

<p>где  <span class="math-tex">\(a_{km}(x) -\)</span>классификатор с позитивным классом <span class="math-tex">\(k\)</span> и негативным классом <span class="math-tex">\(m\)</span>.  Жёстким оно называется потому, что никак не учитывается, насколько уверены были классификаторы в своих предсказаниях.</p>

<p>Если несколько классов для объекта получили одинаковое количество голосов или нужно определить не только класс, но и меру принадлежности, производится «мягкое» голосование: выигрывает тот класс в котором суммарно больше уверены классификаторы.</p>


## EDA

<p>Начнем работу с данными с импорта нужных библиотек:</p>

In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

<p>Прочитаем <a href="https://stepik.org/media/attachments/lesson/537384/suggest_train.csv" rel="noopener noreferrer nofollow">данные</a>:</p>


In [2]:
data = pd.read_csv('suggest_train.csv', index_col=0)
data.head(3)

,title,category_id,name
item_id,,,
335330,Раствор. Товарный бетон от производителя,15,Для дома и дачи|Ремонт и строительство|Стройма...
137480,Детская кроватка,16,"Для дома и дачи|Мебель и интерьер|Кровати, див..."
242690,"Кровать -качалка с ящиком для вещей, два полож...",34,Личные вещи|Товары для детей и игрушки|Детская...


Видим, что данные упорядочены по категориям. Это может привести к ошибке при разбиении данных. Перемешаем данные:

In [3]:
data = data.sample(frac=1).reset_index(drop=True)

Проверим, правильно ли закодированы категории:

In [4]:
np.sort(data.category_id.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53])

In [5]:
data.category_id.min(), data.category_id.max()

(0, 53)

Категории закодированы правильно.

**Баланс классов**

Посмотрим на распределение классов. Большой дисбаланс может вызвать проблемы модели. Например, в топ-3 по вероятности будут попадать нерелевантные категории просто потому, что в датасете их намного больше, чем остальных.

In [6]:
data.groupby('category_id').size().describe()

,0
count,54.000000
mean,370.370370
std,33.178361
min,309.000000
25%,346.000000
50%,361.000000
75%,398.750000
max,447.000000


Видим, что минимальное количество примеров на категорию 309, а максимальное 447. Скорее всего, проблемы не возникнет.



**Примеры заголовков**

Посмотрим на примеры топ-3 наиболее и наименее популярных категорий:

In [7]:
data[data.name.isin(data.name.value_counts()[:3].index)].drop_duplicates(subset='name')

,title,category_id,name
12,Постельное белье 2-спальное с евро-ней N41,27,Для дома и дачи|Мебель и интерьер|Текстиль и к...
25,Вешалка,23,Для дома и дачи|Мебель и интерьер|Другое
92,Набор для покрытия ногтей гель-лаками,37,Личные вещи|Красота и здоровье|Приборы и аксес...


Наименее популярные:

In [8]:
data[data.name.isin(data.name.value_counts()[-3:].index)].drop_duplicates(subset='name')

,title,category_id,name
10,Dell latitude e6540 Новый,1,Бытовая электроника|Ноутбуки
15,Продам телевизор б/у,5,Бытовая электроника|Аудио и видео|Телевизоры и...
60,Пригсинг в нос,40,Личные вещи|Часы и украшения|Ювелирные изделия


Видим, что в бытовой технике могут быть довольно редкие сочетания букв, например "K50AB", есть слова с орфографическими ошибками ("гаршок"). Есть и частые слова, которые встретились в 2 их 6 объявлений.

**Названия категорий**

Посмотрим на названия категорий:

In [9]:
data.name.unique()[:3]

array(['Личные вещи|Товары для детей и игрушки|Товары для кормления',
       'Хобби и отдых|Спорт и отдых|Другое',
       'Бытовая электроника|Телефоны|Nokia'], dtype=object)

Видим, что категории могут находиться на разном уровне иерархии. Посмотрим, насколько сильно может отличаться глубина категории:

In [10]:
data.name.apply(lambda x: len(x.split('|'))).value_counts().sort_index()

,count
name,
2,1406
3,15973
4,2621


Можно подумать, как использовать данные о том, что у классов есть иерархическая структура.

**Статистики по текстам**

Посмотрим, отличаются ли статистически тексты заголовков для разных категорий. Посчитаем количество уникальных символов в заголовке:

In [11]:
data.title.apply(len).describe()

,title
count,20000.000000
mean,25.093300
std,12.474148
min,5.000000
25%,15.000000
50%,23.000000
75%,34.000000
max,50.000000


Посмотрим на на распределение количества слов:

In [12]:
data.title.apply(lambda x: len(x.split())).describe()

,title
count,20000.00000
mean,3.79825
std,1.96368
min,1.00000
25%,2.00000
50%,3.00000
75%,5.00000
max,14.00000


В среднем в заголовках 3-4 слова. Заголовки короткие

Интересно посмотреть на краевые случаи. Давайте выведем примеры, где в заголовок получилось уместить больше 12 слов:

In [13]:
data[data.title.apply(lambda x: len(x.split()) > 12)].head(3)

,title,category_id,name
2871,PS3 Slim + Ps Move + Ps Eye + 3 джойстика + 12...,4,"Бытовая электроника|Игры, приставки и программ..."
6036,HP 17.3 / Core i5 / 4 озу / 2 видео/ как новый,1,Бытовая электроника|Ноутбуки
13127,iPad pro 128 4G + клва + стилус + сумка и так. Д,3,Бытовая электроника|Планшеты и электронные кни...


Все примеры из бытовой электроники. Значит, количество слов может послужить предиктором категории. Посчитаем статистики для категорий:

In [14]:
data['word_count'] = data.title.apply(lambda x: len(x.split()))
data.groupby('name')['word_count'].mean().sort_values().head(3)

,word_count
name,
"Личные вещи|Одежда, обувь, аксессуары|Женская одежда|Платья и юбки",2.576190
Для дома и дачи|Мебель и интерьер|Шкафы и комоды,2.756219
"Личные вещи|Одежда, обувь, аксессуары|Женская одежда|Верхняя одежда",2.824074


Действительно, среднее количество слов для категории отличается в 2 раза.

In [15]:
data.groupby('name')['word_count'].mean().sort_values().tail(3)

,word_count
name,
Хобби и отдых|Велосипеды|Запчасти и аксессуары,5.508600
Бытовая электроника|Телефоны|Аксессуары|Чехлы и плёнки,5.531429
Хобби и отдых|Билеты и путешествия|Путешествия,5.641399


Кажется, что статистики по тексту могут помочь улучшить качество работы классификатора. С другой стороны, нужно хорошенько подумать, стоит ли использовать такие признаки в сервисе?

## Бейзлайн

Разобьем данные на обучающую выборку и валидацию:

In [16]:
train_titles, val_titles, y_train, y_val = train_test_split(
    data['title'],
    data['category_id'],
    random_state=1, stratify=data['category_id']
)

Обычно, в задачах классификации первым бейзлайном является предсказание самым популярным классом. Но когда выборка сбалансирована, а классов много, такой бейзлайн не очень информативен:

In [17]:
(y_val == y_train.mode().values[0]).mean()

0.0224

## CountVectorizer

Для получения матриц признаков преобразуем заголовки с помощью CountVectorizer():

In [18]:
count_vectorizer = CountVectorizer()

X_train = count_vectorizer.fit_transform(train_titles.values)
X_val = count_vectorizer.transform(val_titles.values)

In [19]:
X_train

<15000x17282 sparse matrix of type '<class 'numpy.int64'>'
	with 54626 stored elements in Compressed Sparse Row format>

<p>В данном случае вместо&nbsp;<span class="math-tex">\(15000 \times 17316=259740000\)</span>&nbsp;значений мы храним всего&nbsp;<span class="math-tex">\(54676\)</span> тройки&nbsp;<span class="math-tex">\((i, j, x_j^{(i)})\)</span>. Это&nbsp;1500&nbsp;раз меньше!</p>

<p>Sparse формат не доставляет никаких неудобств &mdash;&nbsp;все полезные функции можно применять к матрицам в разреженном формате. Единственное &laquo;неудобство&raquo;&nbsp;&mdash;&nbsp;нельзя посмотреть&nbsp;на матрицу.<span style="color:#a03881"> </span>Но в данном случае этого делать не нужно, потому что всё, что мы увидим, это матрица&nbsp;из нулей. Не повторяйте этого на практике, на больших&nbsp;выборках преобразованные данные могут не поместиться в оперативную память:</p>

In [20]:
X_train.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

Посмотрим на получившийся словарь:

In [21]:
print('words in vocab: ', len(count_vectorizer.vocabulary_))
count_vectorizer.vocabulary_

words in vocab:  17282


{'samsung': 6039,
 'galaxy': 3799,
 'core': 2929,
 'duos': 3283,
 'sm': 6242,
 'g355h': 3760,
 'ds': 3260,
 'продам': 13865,
 'приложения': 13803,
 'журналам': 9802,
 'насекомые': 12331,
 'коляска': 10724,
 'трость': 15888,
 'шкаф': 16879,
 'столплит': 15384,
 'nan': 5176,
 'гипоаллергенный': 8965,
 '400': 1217,
 'шапка': 16771,
 'вязаная': 8811,
 'ручная': 14501,
 'работа': 14059,
 'маникюрный': 11649,
 'стол': 15369,
 'качели': 10432,
 'люлька': 11522,
 'capella': 2721,
 'трудовая': 15903,
 'книжка': 10603,
 'новая': 12493,
 'телевизор': 15633,
 'рубин': 14439,
 'рабочем': 14066,
 'состоянии': 15136,
 'светодиодные': 14691,
 'лампочки': 11266,
 'лик': 11389,
 'оригинальные': 12777,
 'модули': 12017,
 'xenpak': 7203,
 '10gb': 185,
 'sr': 6348,
 'декортивные': 9311,
 'подушки': 13489,
 'часы': 16651,
 'luminor': 4714,
 'militare': 4968,
 'механика': 11887,
 'acer': 2077,
 'aspire': 2286,
 '4720z': 1320,
 'ноутбук': 12553,
 'rv410': 5942,
 'биология': 8023,
 'учебники': 16157,
 'класс':

Убедимся, что необходимые значения хранятся по индексам из словаря:

In [22]:
count_vectorizer.transform(['Продам продам'])[0, count_vectorizer.vocabulary_['продам']]

2

Видим, что CountVectorizer автоматически приводит слова к нижнему регистру.

Важные параметры

Класс CountVectorizer содержит все нужные параметры для настройки преобразования под вашу задачу:

<pre>
<code class="language-python">CountVectorizer(
    *,
    input='content',
    encoding='utf-8',
    decode_error='strict',
    strip_accents=None,
    lowercase=True,
    preprocessor=None,
    tokenizer=None,
    stop_words=None,
    token_pattern='(?u)\\b\\w\\w+\\b',
    ngram_range=(1, 1),
    analyzer='word',
    max_df=1.0,
    min_df=1,
    max_features=None,
    vocabulary=None,
    binary=False,
    dtype=&lt;class 'numpy.int64'&gt;,
)</code></pre>

<p>Рассмотрим наиболее важные из них:</p>

<ul>
	<li><code>max_df</code>, <code>min_df</code>&nbsp;&mdash;&nbsp;параметры, позволяющие фильтровать слова по порогу встречаемости. Если вы передаёте в параметры действительные числа от 0 до 1, это это максимальная и минимальная доля встречаемости слова. Если целые, то абсолютное значение встречаемости;</li>
	<li><code>ngram_range</code>&nbsp;&mdash;&nbsp;диапазон n-грамм: (1, 3) &mdash;&nbsp;значит, включить в словарь&nbsp;униграммы, биграммы и тригаммы;</li>
	<li><code>analyzer</code>&nbsp;&mdash;&nbsp;определяет, что считать токеном. Есть возможность токенезировать по словам, буквам и буквам с учётом окончаний слов. Может работать особенно эффективно в сочетании с учётом n-грамм;</li>
	<li><code>token_pattern</code>&nbsp;&mdash; регулярное выражение для токенизации.</li>
</ul>


## Модель логистической регрессии

Обучим модель логистической регрессии c параметрами по умолчанию и посчитаем accuracy:

In [23]:
model = LogisticRegression()
model.fit(X_train, y_train)

preds = model.predict(X_val)
accuracy_score(preds, y_val)

0.746

Логистическая регрессия позволяет получить не только предсказанные категории, но и вероятности. Предсказания для одного объекта в нашем случае выглядят как вектор из 54 чисел:

In [24]:
np.round(model.predict_proba(X_val[0]), 3)

array([[0.006, 0.01 , 0.01 , 0.01 , 0.011, 0.014, 0.019, 0.01 , 0.011,
        0.005, 0.009, 0.007, 0.02 , 0.009, 0.013, 0.031, 0.009, 0.151,
        0.013, 0.025, 0.022, 0.021, 0.01 , 0.028, 0.028, 0.009, 0.012,
        0.019, 0.014, 0.019, 0.024, 0.019, 0.012, 0.012, 0.013, 0.007,
        0.011, 0.024, 0.021, 0.015, 0.012, 0.014, 0.011, 0.031, 0.028,
        0.016, 0.018, 0.026, 0.013, 0.024, 0.016, 0.021, 0.021, 0.018]])

Проверим свою интуицию по поводу весов полученной модели. Мы знаем, что слова встречающиеся в разных категориях, не должны получить большой вес в модели. Посмотрим на вес слова 'продам' для разных категорий:

In [25]:
model.coef_[:, count_vectorizer.vocabulary_['продам']]

array([ 0.59974424, -0.1682562 ,  0.51609321, -0.0563477 ,  0.1088072 ,
       -0.40614903,  0.29984701,  0.61040033,  0.00620034, -0.48825412,
        0.98458267,  0.18850967, -0.22362913, -0.18952091, -0.03656822,
       -0.40095667,  0.51637307, -0.84865298,  0.71243572, -0.36803459,
       -0.46992469,  0.21785386,  0.4858873 , -0.04136437, -0.36302849,
        0.25945184,  0.42459152, -0.04835539,  0.18409748,  0.55801259,
       -0.34234967,  0.22968516,  0.45163617,  0.99778192,  0.59886766,
        0.36963754, -0.31974293, -0.1306174 ,  0.65095308,  0.38454104,
       -0.15146212, -0.10910652, -0.15377463, -1.0667379 , -0.65276616,
       -0.25417019,  0.1848521 , -0.49626756, -1.363402  ,  0.10618286,
        0.48282162, -0.88010467, -0.20371968, -0.89658331])

Слова, по которым можно определить категорию, наоборот, должны получить большой вес. Посмотрим на вес для слова 'iphone' для разных категорий:

In [26]:
model.coef_[:, count_vectorizer.vocabulary_['iphone']]

array([ 6.26419331,  0.23140296, -0.18257798, -0.24478176, -0.17583049,
       -0.17751557,  1.49166034, -0.1662587 , -0.21799909,  3.37864231,
       -0.17641448, -0.12768858,  0.69694238, -0.14543015, -0.29694921,
       -0.37764221, -0.14659407, -0.33601097, -0.2071416 , -0.30925772,
       -0.31683976, -0.34425558, -0.16357921, -0.36319612, -0.32667711,
       -0.13229036, -0.18523416, -0.30423574, -0.21207474, -0.26367094,
       -0.32533283, -0.26771774, -0.18291049, -0.18234216, -0.20519099,
       -0.11674671, -0.16436543, -0.36792387, -0.30010765, -0.20161636,
       -0.27270807, -0.23629622, -0.20172193, -0.33890369, -0.33136932,
       -0.21271924, -0.25709596, -0.36632702, -0.19239294, -0.34991924,
       -0.22439452, -0.26650067, -0.30140482, -0.29668714])

Действительно, вес при этом слове для некоторых категорий очень большой. Ведь если это слово есть в заголовке, скорее всего, правильной категорией будет что-то начинающееся на 'Бытовая электроника|Телефоны|'. Проверим, так ли это:

In [27]:
cat_names = data.set_index('category_id')['name'].to_dict()
top_iphone_cats = np.argsort(model.coef_[:, count_vectorizer.vocabulary_['iphone']])[-3:]
[cat_names[c] for c in top_iphone_cats]

['Бытовая электроника|Телефоны|Другие марки',
 'Бытовая электроника|Телефоны|Аксессуары|Чехлы и плёнки',
 'Бытовая электроника|Телефоны|iPhone']

**Ошибки модели**

Посмотрим, в каких категориях чаще всего происходят ошибки. Для этого создадим удобный dataframe:

In [28]:
models_erros = pd.DataFrame({
    'title': val_titles.values,
    'preds_category': [cat_names[cat] for cat in preds],
    'true_category': [cat_names[cat] for cat in y_val],
    'preds_ok': preds == y_val
})

Выведем 3 случайные ошибки модели:

In [29]:
models_erros[~models_erros['preds_ok']].sample(3)

,title,preds_category,true_category,preds_ok
10740,Столбы деревянные 3.15 метра длиной,Для дома и дачи|Мебель и интерьер|Другое,Для дома и дачи|Ремонт и строительство|Стройма...,False
11877,Статуэтка Аленький цветочек,Для дома и дачи|Мебель и интерьер|Предметы инт...,Хобби и отдых|Коллекционирование|Другое,False
4448,Саундбар philips hts8100,Бытовая электроника|Аудио и видео|Телевизоры и...,"Бытовая электроника|Аудио и видео|Акустика, ко...",False


Видим, что в первых двух строчках ошибки, скорее всего, произошли по причине опечаток. Нужно подумать над тем, как повысить устойчивость модели к опечаткам и ошибкам. Этим вам предстоит заняться в домашнем задании.

На этом анализ ошибок, конечно же, заканчивать не стоит — но мы не будем продолжать это делать здесь, а оставим в виде упражнения.

## FastText

Импортируем gensim – библиотеку, с помощью которой будем обучать модель FastText.

In [53]:
import gensim
from gensim.models.callbacks import CallbackAny2Vec

In [54]:
print(gensim.__version__)

4.3.3


Прочитаем данные и выведем первые несколько строк:

In [55]:
fast_text_train_data = pd.read_csv('unlabeled_data.csv',  index_col=0)
print(fast_text_train_data.shape)
fast_text_train_data.head()

(200000, 2)


,item_id,title
30000,377939,1 рубль царские 10 штук
30001,384779,"Напольный насос giyo высокого давления, с мано..."
30002,276889,Sony Xperia m2 Dual
30003,155718,Zoom B2.1U+ AC Процессор эффектов для бас-гитары
30004,260705,Люстра Favourite petite 1574-7P


Данные для обучения word2vec — это просто тексты или любые другие наборы последовательностей токенов, для которых можно предположить справедливость дистрибутивной гипотезы. Токенизируем тексты:

In [56]:
import re

In [57]:
WORD_PATTERN = '(?u)\\b\\w\\w+\\b'  # паттерн для токенизации возьмем из векторайзеров sklearn
reg_exp = re.compile(pattern=WORD_PATTERN)  # скомпилируем регулярное выражение
sentences = [reg_exp.findall(s.lower()) for s in fast_text_train_data.title]

Выведем результат токенизации:

In [58]:
sentences[:3]

[['рубль', 'царские', '10', 'штук'],
 ['напольный', 'насос', 'giyo', 'высокого', 'давления', 'манометр'],
 ['sony', 'xperia', 'm2', 'dual']]

Обучим FastText с данными параметрами:

In [59]:
from gensim.models import FastText

fasttext_model = FastText(
    sentences=sentences,      # корпус токенизированных названий товаров
    vector_size=90,           # размерность эмбеддингов
    window=3,                 # размер окна контекста
    min_count=5,              # минимальное количество вхождений слова
    sg=1,                     # используем Skip-gram
    negative=5,               # отрицательное семплирование
    workers=4                 # параллелизация
)

fasttext_model.build_vocab(sentences)
fasttext_model.train(
    corpus_iterable=sentences,
    total_examples=fasttext_model.corpus_count,
    epochs=50,
    compute_loss=True
)

(29076961, 36711850)

Попытаемся понять, получилось ли обучиться. Выведем векторы, близкие к вектору слова 'продать';

In [60]:
fasttext_model.wv.similar_by_word('продать')

[('продажу', 0.819695770740509),
 ('продаём', 0.8134431838989258),
 ('продадим', 0.8084778785705566),
 ('продаеться', 0.788177490234375),
 ('продам', 0.7860618233680725),
 ('продаю', 0.774895191192627),
 ('продаж', 0.7703099250793457),
 ('продаже', 0.7602257132530212),
 ('продаем', 0.7462074756622314),
 ('продается', 0.722420334815979)]

Посмотрим на ещё одно слово. Интересно, что можно поискать слова близкие к слову 'айфон' и тем самым получить написание этого слова на английском:

In [61]:
fasttext_model.wv.similar_by_word('айфон')

[('айфон4', 0.9433593153953552),
 ('айфон6', 0.8970489501953125),
 ('айфоны', 0.8271320462226868),
 ('iphone', 0.8000407218933105),
 ('iphone5s', 0.7919464707374573),
 ('iphone4s', 0.7907669544219971),
 ('iphone4', 0.7872167229652405),
 ('айфона', 0.7839094400405884),
 ('iphone5', 0.7792999744415283),
 ('айфонов', 0.7551853060722351)]

Напишем класс, которые позволит там получать векторы для заголовков, как сумму векторов входящих в него слов:

In [62]:
class FastTextTransformer:

    def __init__(self, fasttext_model, word_pattern):

        self.fasttext_model = fasttext_model
        self.word_pattern = word_pattern
        self.re = re.compile(pattern=self.word_pattern)

    def fit(self, X):
        return self

    def transform(self, X):
        X_transformed = np.zeros((len(X), self.fasttext_model.wv.vector_size))
        for i, title in enumerate(X):
            tokens = self.re.findall(title.lower())
            title_vector = np.zeros((self.fasttext_model.wv.vector_size,))

            count = 0
            for token in tokens:
                if token in self.fasttext_model.wv.key_to_index:
                    title_vector += self.fasttext_model.wv.get_vector(token)
                    count += 1
            # Усредняем векторы
            if count > 0:
                title_vector /= count
            X_transformed[i] = title_vector
        return X_transformed



Трансформируем с помощью него обучающую выборку и обучим логистическую регрессию:

In [63]:
fast_text_transformer = FastTextTransformer(fasttext_model=fasttext_model, word_pattern=WORD_PATTERN)

train_fast_text = fast_text_transformer.transform(train_titles.values)
val_fast_text = fast_text_transformer.transform(val_titles.values)

model_on_fast_text = LogisticRegression(solver='liblinear')
model_on_fast_text.fit(train_fast_text, y_train)

LogisticRegression(solver='liblinear')

Оценим качество классификатора:

In [64]:
preds = model_on_fast_text.predict(val_fast_text)
accuracy_score(preds, y_val)

0.7646

Так как мы оцениваем suggest, нам важны именно первые рекомендованные выдачи. Поэтому посчитаем **accuracy top 3**

In [65]:
prebs = model_on_fast_text.predict_proba(val_fast_text)
top_3_preds = np.argsort(prebs, axis=1)[:, -3:]
accuracy_at_3 = np.mean([y_val.iloc[i] in top_3_preds[i] for i in range(len(y_val))])

print(f"Accuracy @3: {accuracy_at_3}")

Accuracy @3: 0.8902
